In [1]:
from pycocotools.coco import COCO
import os
import cv2
import time
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random

In [2]:
# Annotation Files
train_ann = "/home/ani/Documents/datasets/coco/annotations/instances_train2017.json"
val_ann = "/home/ani/Documents/datasets/coco/annotations/instances_val2017.json"

train_img_dir = '/home/ani/Documents/datasets/coco/train2017/'
val_img_dir = '/home/ani/Documents/datasets/coco/val2017/'

train_dest_segmentation_masks = '/home/ani/Documents/datasets/coco/segmentations/train'
val_dest_segmentation_masks = '/home/ani/Documents/datasets/coco/segmentations/val'

train_dest_cropped_images = '/home/ani/Documents/datasets/coco/cropped_images/train'
val_dest_cropped_images = '/home/ani/Documents/datasets/coco/cropped_images/val'

In [3]:
# Hyperparameters
DIMS = 100, 100
MIN_IMAGE_SIZE = 30
PADDING_CHOICES = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

In [4]:
# Initialize the COCO objects
train_coco = COCO(train_ann)
val_coco = COCO(val_ann)

loading annotations into memory...
Done (t=9.21s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!


In [5]:
def letterbox_image(image, size):
    ih, iw = image.shape[:2]
    w, h = size
    scale = min(w / iw, h / ih)

    # Get the new scaled width and height
    nw = int(scale * iw)
    nh = int(scale * ih)

    # Resize Image based on it's initial size maintaining original aspect ratio
    if nw > iw or nh > ih:
        image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
    else:
        image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_AREA)

    # Create a blank template image
    new_image = np.zeros((h, w, 3), np.uint8)

    # Calculate the offsets
    dx = (w - nw) // 2
    dy = (h - nh) // 2

    new_image[dy : dy + nh, dx : dx + nw] = image

    return new_image


def get_cropped_rois(coco_object, index, filtered_cat_id, filtered_img_id, img_dir):
    # Load Image
    img_meta = coco_object.loadImgs(filtered_img_id[index])
    img_path = os.path.join(img_dir, img_meta[0]['file_name'])
    img = cv2.imread(img_path)
    height, width = img.shape[:2]

    # Load Annotations
    annIds = coco_object.getAnnIds(imgIds=img_meta[0]['id'], catIds=filtered_cat_id, iscrowd=False)
    anns = coco_object.loadAnns(annIds)
    
    # Generate Mask
    anns_mask = np.zeros((height, width), dtype=np.uint8)
    for i, ann in enumerate(anns):
        # Extract Mask
        anns_mask = coco_object.annToMask(ann)*ann['category_id']
        anns_mask = (anns_mask / anns_mask.max() * 255).astype(np.uint8)
        anns_mask = cv2.cvtColor(anns_mask, cv2.COLOR_GRAY2BGR)
        
        # Extract Coordinates
        x, y, w, h = ann["bbox"]
        xmin, ymin, xmax, ymax = int(x), int(y), int(x + w), int(y + h)
        
        # Filter BBoxes < MIN_IMAGE_SIZE Pixels in size 
        if min(w, h) < MIN_IMAGE_SIZE:
            continue
        
        # Add Some Padding
        padding_percent = random.choice(PADDING_CHOICES)
        xmin = max(0, int(xmin - padding_percent * w))
        ymin = max(0, int(ymin - padding_percent * h))
        xmax = min(width, int(xmax + padding_percent * w))
        ymax = min(height, int(ymax + padding_percent * h))
        
        # Exctract ROI
        cropped_mask = anns_mask[ymin: ymax, xmin: xmax]
        cropped_image = img[ymin: ymax, xmin: xmax]
        
        # Letterbox Image
        cropped_mask = letterbox_image(cropped_mask, DIMS)
        cropped_image = letterbox_image(cropped_image, DIMS)
        
        filename = f"{img_meta[0]['file_name'][:-4]}_{i}.jpg"
        
        yield filename, cropped_image, cropped_mask


def write_cropped_rois(
        coco_object,
        index, 
        filtered_cat_id, 
        filtered_img_id, 
        dest_image_folder, 
        dest_mask_folder, 
        img_dir
    ):

    if not os.path.isdir(dest_image_folder):
        os.makedirs(dest_image_folder)
        
    if not os.path.isdir(dest_mask_folder):
        os.makedirs(dest_mask_folder)
        
    generator = get_cropped_rois(coco_object, index, filtered_cat_id, filtered_img_id, img_dir)
    for filename, image, mask in generator:
        # Generate Paths
        dest_image_path = os.path.join(dest_image_folder, filename)
        dest_mask_path = os.path.join(dest_mask_folder, filename)
        
        # Write Images & Masks
        cv2.imwrite(dest_image_path, image)
        cv2.imwrite(dest_mask_path, mask)


def write_train_masks(train_coco, train_dest_image_folder, train_dest_mask_folder, classes=['person']):
    print("Processing Train Images")
    time.sleep(0.5)
    filtered_cat_id = train_coco.getCatIds(classes)
    filtered_img_id = train_coco.getImgIds(catIds=filtered_cat_id) 
    
    for i in tqdm(range(len(filtered_img_id))):
        write_cropped_rois(
            train_coco, i,
            filtered_cat_id, filtered_img_id, 
            train_dest_image_folder, train_dest_mask_folder, 
            train_img_dir
        )
        
    print("Train Segmentation Masks Processing Complete")


def write_val_masks(val_coco, val_dest_image_folder, val_dest_mask_folder, classes=['person']):
    print("Processing Val Images")
    time.sleep(0.5)
    filtered_cat_id = val_coco.getCatIds(classes)
    filtered_img_id = val_coco.getImgIds(catIds=filtered_cat_id) 
    
    for i in tqdm(range(len(filtered_img_id))):
        write_cropped_rois(
            val_coco, i, 
            filtered_cat_id, filtered_img_id, 
            val_dest_image_folder, val_dest_mask_folder, 
            val_img_dir
        )
        
    print("Val Segmentation Masks Processing Complete")

In [6]:
write_val_masks(
    val_coco, 
    val_dest_cropped_images,
    val_dest_segmentation_masks, 
    classes=None
)
write_train_masks(
    train_coco, 
    train_dest_cropped_images,
    train_dest_segmentation_masks, 
    classes=None
)

Processing Val Images


100%|██████████| 5000/5000 [01:20<00:00, 62.09it/s]


Val Segmentation Masks Processing Complete
Processing Train Images


100%|██████████| 118287/118287 [34:48<00:00, 56.63it/s] 

Train Segmentation Masks Processing Complete
